### Initial Setup
Firstly, we will import the lsmgridtrack core module and create a tracker object with default options.

In [1]:
import lsmgridtrack as lsm

t = lsm.tracker()

Since we didn't provide an options or config keyword argument this tracker object has the default options. We can view these.

In [2]:
print(t.options)

{'Image': {'spacing': [1.0, 1.0, 1.0], 'resampling': [1.0, 1.0, 1.0]}, 'Grid': {'origin': False, 'spacing': False, 'size': False, 'crop': False}, 'Registration': {'method': 'BFGS', 'iterations': 100, 'sampling_fraction': 0.05, 'sampling_strategy': 'RANDOM', 'usemask': False, 'landmarks': False, 'shrink_levels': [1], 'sigma_levels': [0.0]}}


These are a custom class built on the normal Python dictionary, but with immutable keys. If we try to introduce a new key, an error will be raised. This will help prevent spelling typos from causing runtime bugs.

In [3]:
t.options['foo'] = True

KeyError: 'foo is not an acceptable key.'

### Indicate paths to image files
We didn't provide a reference or deformed image to register. Let's do that now with the two images included in the test module.

In [4]:
from lsmgridtrack.test import data

# Names of available images
print(data.get_image_names())


['reference - 1 layer', '10 percent strain - 1 layer', 'reference - 2 layers', '10 percent strain - 2 layers']


In [5]:
# path to reference image
reference = data.get_image('reference - 2 layers')

# path to deformed image
deformed = data.get_image('10 percent strain - 2 layers')

# assign these image paths to tracker object
t.reference_path = reference
t.deformed_path = deformed


Now, we have images to analyze, but the default options are not correct for these. Let's modify these directly.

In [6]:
# Change the image spacing
t.options['Image']['spacing'] = [0.5, 0.5, 1.0]
# Change the grid origin, spacing, and size
t.options['Grid']['origin'] = [69, 72, 5]
t.options['Grid']['spacing'] = [20, 20, 10]
t.options['Grid']['size'] = [20, 20, 3]

# Set the registration method to BFGS
t.options['Registration']['method'] = 'BFGS'


#See the options are now changed
print(t.options)

{'Image': {'spacing': [0.5, 0.5, 1.0], 'resampling': [1.0, 1.0, 1.0]}, 'Grid': {'origin': [69, 72, 5], 'spacing': [20, 20, 10], 'size': [20, 20, 3], 'crop': False}, 'Registration': {'method': 'BFGS', 'iterations': 100, 'sampling_fraction': 0.05, 'sampling_strategy': 'RANDOM', 'usemask': False, 'landmarks': False, 'shrink_levels': [1], 'sigma_levels': [0.0]}}


### Running the registration and analysis
Now, let's perform the registration and post-processing.

In [7]:
t.execute()

... Starting Deformable Registration
... ... Finding optimal BSpline transform
... ... Elapsed Iterations: 0
... ... Current Metric Value: -4.49300E-02
... ... Elapsed Iterations: 0
... ... Current Metric Value: -5.40839E-02
... ... Elapsed Iterations: 0
... ... Current Metric Value: -7.82703E-02
... ... Elapsed Iterations: 0
... ... Current Metric Value: -7.82703E-02
... ... Elapsed Iterations: 1
... ... Current Metric Value: -5.33780E-02
... ... Elapsed Iterations: 1
... ... Current Metric Value: -7.28831E-03
... ... Elapsed Iterations: 1
... ... Current Metric Value: -8.40190E-02
... ... Elapsed Iterations: 1
... ... Current Metric Value: -8.40190E-02
... ... Elapsed Iterations: 2
... ... Current Metric Value: -8.56601E-02
... ... Elapsed Iterations: 2
... ... Current Metric Value: -8.56601E-02
... ... Elapsed Iterations: 3
... ... Current Metric Value: -8.78699E-02
... ... Elapsed Iterations: 3
... ... Current Metric Value: -8.78699E-02
... ... Elapsed Iterations: 4
... ... Current

We did not provide any initialization to the registration algorithm in the above execution. If we provide the indices of the 8 grid corners (ordered counter-clockwise) in the deformed image, the registration can be better initialized. Let's see if this initialization changes the convergence behaviour. We determined these voxel indices using ImageJ.

In [8]:
t.options['Registration']['landmarks'] = [[72, 81, 5],
                                          [71, 467, 5],
                                          [457, 468, 5],
                                          [455, 82, 5],
                                          [71, 80, 20],
                                          [72, 468, 20],
                                          [458, 466, 20],
                                          [457, 80, 20]]

And re-executing the registration.

In [9]:
t.execute()

... Starting Deformable Registration
... ... Finding optimal BSpline transform
... ... Elapsed Iterations: 0
... ... Current Metric Value: -2.32286E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -2.62576E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -3.85003E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -3.85003E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -1.67176E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -3.80853E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -5.01466E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -5.01466E-01
... ... Elapsed Iterations: 2
... ... Current Metric Value: -5.62287E-01
... ... Elapsed Iterations: 2
... ... Current Metric Value: -5.62287E-01
... ... Elapsed Iterations: 3
... ... Current Metric Value: -5.52476E-01
... ... Elapsed Iterations: 3
... ... Current Metric Value: -6.67195E-01
... ... Elapsed Iterations: 3
... ... Current

We converged in less iterations of BFGS than the uninitialized registration. The final metric value (negative cross-correlation) was quite close. This suggests the objective function may be near convex since the determined minima are nearly equal; although, this cannot be proven. Qualitative inspection of the two results suggests these particular images can be registered well without initialization. The reader is encouraged to do this inspection by outputting results from each execution.

### Saving the results
We can write the results in different formats such as a VTK image,

In [10]:
t.writeResultsAsVTK('example1')

... Saving Results to example1.vti


an Excel workbook,

In [11]:
t.writeResultsAsExcel('example1')

... Saving Results to example1.xlsx


and a NumPy binary.

In [12]:
t.writeResultsAsNumpy('example1')

... Saving file as numpy archive example1.npz


### Saving the 3D images for later visualization
To view the original images in the open source 3D visualization software, ParaView, we can save the images as a VTK image.

In [13]:
# Write the reference image to VTK image
t.writeImageAsVTK(t.ref_img, 'reference')

# WRite the deformed image to VTK image
t.writeImageAsVTK(t.def_img, 'deformed')

... Saving Image to reference.vti
... Saving Image to deformed.vti
